# Lectura de datos
## Funcion para gregar el set de datos 

In [4]:
import numpy as np
import os
import gzip

def cargar_set(ruta, tipo='train'):

  ruta_categorias = os.path.join(ruta, '%s-labels-idx1-ubyte.gz' %tipo)
  ruta_imagenes = os.path.join(ruta, '%s-images-idx3-ubyte.gz' %tipo)

  with gzip.open(ruta_categorias, 'rb') as rut_cat:
    etiquetas = np.frombuffer(rut_cat.read(), dtype = np.uint8, offset = 8)

  with gzip.open(ruta_imagenes, 'rb') as rut_imgs:
    imagenes = np.frombuffer(rut_imgs.read(), dtype = np.uint8, offset = 16).reshape(len(etiquetas), 784)
  
  return imagenes, etiquetas



## Obtener acceso a nuestro set de datos en drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

ruta = 'gdrive/My Drive/Colab Notebooks/fashion_mnist_data'

X_train, Y_train = cargar_set(ruta, tipo='train')
X_test, Y_test = cargar_set(ruta, tipo='test')

Mounted at /content/gdrive


## Reshape de los datos

In [6]:
X_train = X_train[0:59904,:]
X_test =  X_test[0:9984,:]
Y_train = Y_train[0:59904]
Y_test = Y_test[0:9984]

X_train = np.reshape(X_train, (59904,28,28,1))
X_test = np.reshape(X_test, (9984,28,28,1))


## Importando Tensorflow 2

In [7]:
%tensorflow_version 2.x
import tensorflow as tf

print('version de Tensorflow: ' + tf.__version__)


version de Tensorflow: 2.4.1


# Creción del modelo 

In [8]:
tf.random.set_seed(200)

modelo = tf.keras.models.Sequential()

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(64,(5,5), padding='same', activation='elu'))
modelo.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(128,(5,5), padding='same', activation='elu'))
modelo.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.BatchNormalization(input_shape=X_train.shape[1:]))
modelo.add(tf.keras.layers.Conv2D(256,(5,5), padding='same', activation='elu'))
modelo.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
modelo.add(tf.keras.layers.Dropout(0.25))

modelo.add(tf.keras.layers.Flatten())
modelo.add(tf.keras.layers.Dense(256)) #Dense indica las neuronas a utilizar
modelo.add(tf.keras.layers.Activation('elu'))
modelo.add(tf.keras.layers.Dropout(0.5))
modelo.add(tf.keras.layers.Dense(10))
modelo.add(tf.keras.layers.Activation('softmax'))

#imprimimos la informacion del modelo
modelo.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       204928    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)        

# Identificando Gpu

In [13]:
nombre_gpu = tf.test.gpu_device_name()
if nombre_gpu != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('GPU encontrada: {}'.format(nombre_gpu))






GPU encontrada: /device:GPU:0


# Compilando el modelo

In [18]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento 1 con GPU

In [19]:
import timeit

def entrenamiento_gpu():
  with tf.device('/device:GPU:0'):
    modelo.fit(X_train,Y_train,validation_data=(X_test,Y_test),batch_size=128, epochs=2, verbose=1)
  return None

gpu_time = timeit.timeit('entrenamiento_gpu()', number=1, setup='from __main__ import entrenamiento_gpu')


Epoch 1/2
468/468 [==============================] - 41s 17ms/step - loss: 1.0138 - accuracy: 0.7087 - val_loss: 0.4277 - val_accuracy: 0.8527
Epoch 2/2
468/468 [==============================] - 8s 16ms/step - loss: 0.4141 - accuracy: 0.8546 - val_loss: 0.3611 - val_accuracy: 0.8788


In [20]:
print('Tiempo de entrenamiento ' + str(gpu_time) + ' Segundos')

Tiempo de entrenamiento 48.56197030300018 Segundos
